<a href="https://colab.research.google.com/github/poojagolande256-ui/Learning_Studio/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")


Step 1 Data Ingestion

In [2]:
url1="https://raw.githubusercontent.com/poojagolande256-ui/Learning_Studio/refs/heads/main/Restaurant_Reviews.tsv"

In [3]:
import pandas as pd
df=pd.read_csv(url1,sep="\t")
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


Step 2 Text Preprocessing

In [5]:
import re
text="wOw.......???pune Location Outstanding!!!!!!"
text=text.lower()
text=re.sub(r"[^\w\s]","",text)
text=text.strip()
print(text)

wowpune location outstanding


In [9]:
def clean_text(text: str) ->str:
  text=text.lower()
  text=re.sub(r"[^\w\s]","",text)
  text=text.strip()
  return text

In [10]:
clean_text("how are you.........?????$$%")

'how are you'

In [14]:
df["clean"]=df["Review"].apply(clean_text)
df["clean"].head()

,clean
0,wow loved this place
1,crust is not good
2,not tasty and the texture was just nasty
3,stopped by during the late may bank holiday of...
4,the selection on the menu was great and so wer...


Step 3 Separate x & y

In [16]:
x=df["clean"]
y=df["Liked"]

Step 4 Apply TF-IDF

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tf.fit_transform(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9754 stored elements and shape (1000, 2067)>

In [19]:
x_pre=tf.transform(x).toarray()

In [21]:
x_pre.shape

(1000, 2067)

Step 5 Reduce the dimesnsions using SVD - Singular Value Decomposition
reduce the columns from 2067 - 300

In [25]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=300)
svd.fit_transform(x_pre)

array([[ 0.11457856,  0.19021732,  0.29860512, ...,  0.01506239,
        -0.00481141,  0.00798915],
       [ 0.20041515,  0.06219205,  0.10386267, ...,  0.0240411 ,
         0.04441308,  0.08610058],
       [ 0.248602  , -0.05947125, -0.09197253, ...,  0.02570916,
         0.03389054,  0.00761634],
       ...,
       [ 0.25745938,  0.21354302, -0.25268859, ...,  0.00688392,
        -0.00806574,  0.01067527],
       [ 0.21755074,  0.04422347, -0.07900023, ..., -0.04502109,
        -0.0356482 , -0.01203185],
       [ 0.22278543,  0.02689928, -0.02221584, ..., -0.01148399,
        -0.01406079, -0.0551153 ]])

In [26]:
x_pre_red=svd.transform(x_pre)
x_pre_red[0:5]

array([[ 0.11457856,  0.19021732,  0.29860512, ...,  0.01506239,
        -0.00481141,  0.00798915],
       [ 0.20041515,  0.06219205,  0.10386267, ...,  0.0240411 ,
         0.04441308,  0.08610058],
       [ 0.248602  , -0.05947125, -0.09197253, ...,  0.02570916,
         0.03389054,  0.00761634],
       [ 0.10419317, -0.00728011, -0.00104468, ...,  0.05421066,
         0.06045865,  0.03432847],
       [ 0.40883954, -0.19004255, -0.03852557, ..., -0.00203189,
         0.00443506,  0.01848729]])

In [27]:
x_pre_red.shape

(1000, 300)

Step 6 Train test split

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_pre_red,y,test_size=0.2,random_state=42)


Step 7 Build neural network

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [30]:
model=Sequential(
    [
        Input(shape=(300,)),
        Dense(128,activation="relu",kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(64,activation="relu",kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(1,activation="sigmoid")
    ]
)

Step 6 Compile model

In [31]:
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["auc"])